**Problem Statement**
The primary goal was to identify fraudulent transactions from a vast transaction history dataset. We intended to develop a reliable predictive model that could classify transactions as either legitimate or fraudulent with high precision.

**Data Handling and Exploration**
We began by loading the dataset from Google Drive into a pandas dataframe. It involved processing large volumes of data, and thus, we utilized PySpark, a large-scale data processing tool.
Next, we performed exploratory data analysis (EDA) on our dataset. We used various visualization tools to understand the underlying patterns and distributions in our data. In particular, we concentrated on the target variable 'isFraud'. We dropped unnecessary columns, such as 'isFlaggedFraud', and calculated some additional columns like the difference between old and new balance for origin and destination.

**Feature Engineering**
For our model to comprehend the data, we had to convert categorical data into numerical forms using one-hot encoding. This process helped ensure our machine learning model wouldn't misinterpret the categorical data as ordinal data.

Additionally, we scaled our numerical features to standardize their range. This step is crucial, especially when our dataset contains features with different scales, as it helps avoid any particular feature from dominating others.

Then we Created new features 'diffOrigBal', 'diffDestBal', and 'transferAmount'. Then, they have scaled the features using StandardScaler from the PySpark ML library.

In [ ]:
df = df.withColumn('diffOrigBal', col('oldbalanceOrg') - col('newbalanceOrig'))
df = df.withColumn('diffDestBal', col('oldbalanceDest') - col('newbalanceDest'))
df = df.withColumn('transferAmount', col('oldbalanceOrg') - col('newbalanceDest'))
vector_assembler = VectorAssembler(inputCols=['type_encoded', 'amount', 'diffOrigBal', 'diffDestBal', 'transferAmount'], outputCol='unscaled_features')
scaler = StandardScaler(inputCol='unscaled_features', outputCol='features')
pipeline = Pipeline(stages=[vector_assembler, scaler])
df = pipeline.fit(df).transform(df)

**Modeling and Evaluation**
We trained two models, Logistic Regression and Random Forest Classifier. Both of these models serve as robust methods for tackling classification problems. They were trained on the training set and evaluated on the test set using the Area Under the Curve (AUC) metric.

In [ ]:
lr_engineered_model = lr_engineered.fit(va_engineered_scaled)
lr_engineered_preds = lr_engineered_model.transform(scaler_model.transform(va_engineered.transform(test)))
lr_engineered_evaluator = BinaryClassificationEvaluator(labelCol='isFraud')
lr_engineered_auc = lr_engineered_evaluator.evaluate(lr_engineered_preds)

rf_engineered_model = rf_engineered.fit(va_engineered_scaled)
rf_engineered_preds = rf_engineered_model.transform(scaler_model.transform(va_engineered.transform(test)))
rf_engineered_evaluator = BinaryClassificationEvaluator(labelCol='isFraud')
rf_engineered_auc = rf_engineered_evaluator.evaluate(rf_engineered_preds)

print('Logistic Regression with Engineered Features AUC:', lr_engineered_auc)
print('Random Forest with Engineered Features AUC:', rf_engineered_auc)


**Findings and Results**

Our models performed considerably well in predicting fraudulent transactions. We found that the Random Forest Classifier provided a slightly better performance compared to Logistic Regression due to its ensemble nature, robustness to outliers, and ability to model non-linear decision boundaries.

The use of machine learning in this project allowed us to detect potential fraudulent transactions effectively, which could significantly reduce the financial losses related to such fraudulent activities. This project showed the tremendous potential of applying machine learning techniques to improve security measures in the financial industry.